<a href="https://colab.research.google.com/github/matveevk/spot-the-bot/blob/main/matveev/coursework_glove_plus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gensim
import gensim.downloader
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from sklearn.metrics import mean_squared_error as mse
from tqdm.notebook import tqdm
from gensim.parsing.preprocessing import (
    preprocess_string,
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    strip_numeric
)

import pandas as pd
import seaborn as sns

from sklearn.decomposition import IncrementalPCA    # inital reduction
from sklearn.manifold import TSNE                   # final reduction

# Предобученный w2v (glove)

## Загрузка модели + векторизирование текста

In [ ]:
list(gensim.downloader.info()['models'].keys())

['fasttext-wiki-news-subwords-300',
 'conceptnet-numberbatch-17-06-300',
 'word2vec-ruscorpora-300',
 'word2vec-google-news-300',
 'glove-wiki-gigaword-50',
 'glove-wiki-gigaword-100',
 'glove-wiki-gigaword-200',
 'glove-wiki-gigaword-300',
 'glove-twitter-25',
 'glove-twitter-50',
 'glove-twitter-100',
 'glove-twitter-200',
 '__testing_word2vec-matrix-synopsis']

In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


In [ ]:
glove_vectors.most_similar(positive=['king', 'woman'], negative=['queen'])

[('man', 0.857354462146759),
 ('who', 0.8164044618606567),
 ('father', 0.8128569722175598),
 ('death', 0.7859841585159302),
 ('another', 0.7801331281661987),
 ('whose', 0.7746530175209045),
 ('old', 0.7679885029792786),
 ('victim', 0.7661791443824768),
 ('him', 0.7624204158782959),
 ('thought', 0.7570509910583496)]

In [ ]:
glove_vectors.most_similar(positive=['king', 'woman'], negative=['man'])

[('queen', 0.8523603677749634),
 ('throne', 0.7664334177970886),
 ('prince', 0.759214460849762),
 ('daughter', 0.7473883032798767),
 ('elizabeth', 0.7460220456123352),
 ('princess', 0.7424569725990295),
 ('kingdom', 0.7337411642074585),
 ('monarch', 0.7214490175247192),
 ('eldest', 0.7184861898422241),
 ('widow', 0.7099430561065674)]

In [ ]:
from gensim.test.utils import datapath

results = glove_vectors.evaluate_word_analogies(datapath('questions-words.txt'))
results[0]

0.463717540798522

In [ ]:
for i in range(1, len(results[1])):
    y = []
    for test in results[1][i]['correct'] + results[1][i]['incorrect']:
        try:
            best_outputs = [z[0] for z in glove_vectors.most_similar([test[0].lower(), test[3].lower()], [test[1].lower()])[:5]]
            y.append(test[2].lower() in best_outputs)
            # print(closest, test[2])
            # y.append(closest == test[2].lower())
        except:
            pass
    print(np.mean(y), len(y), results[1][i]['section'])

0.7701149425287356 4524 capital-world
0.24133663366336633 808 currency
0.10903931901094446 2467 city-in-state
0.8754940711462451 506 family
0.2913306451612903 992 gram1-adjective-to-adverb
0.20935960591133004 812 gram2-opposite
0.7394894894894894 1332 gram3-comparative
0.624777183600713 1122 gram4-superlative
0.7007575757575758 1056 gram5-present-participle
0.957473420888055 1599 gram6-nationality-adjective
0.6051282051282051 1560 gram7-past-tense
0.8085585585585585 1332 gram8-plural
0.725287356321839 870 gram9-plural-verbs
0.6096997690531177 19485 Total accuracy


In [ ]:
glove_vectors.most_similar('moscow')

[('kiev', 0.8409875631332397),
 ('prague', 0.8244465589523315),
 ('warsaw', 0.8230167031288147),
 ('russia', 0.820088803768158),
 ('russian', 0.8137035965919495),
 ('belgrade', 0.7946885228157043),
 ('berlin', 0.7849311232566833),
 ('minsk', 0.7801254987716675),
 ('kremlin', 0.7750862836837769),
 ('tehran', 0.7637045383453369)]

In [ ]:
vocab = list(glove_vectors.index2entity)
X = glove_vectors.wv[vocab]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


NameError: ignored

In [ ]:
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X[:2100, :])
sns.set_style("white")

df = pd.DataFrame(X_tsne, index=vocab[:2100], columns=['x', 'y'])

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(1, 1, 1)

cols = ['blue'] * len(X_tsne)
for spec in ['king', 'queen', 'man', 'woman', 'can', 'must', 'should', 'feel', 'hurt']:
    cols[vocab.index(spec)] = 'red'

ax.scatter(df['x'], df['y'], c=cols)


ax.annotate('king', X_tsne[vocab.index('king')], size=20, )
ax.annotate('queen', X_tsne[vocab.index('queen')], size=20)
ax.annotate('man', X_tsne[vocab.index('man')], size=20)
ax.annotate('woman', X_tsne[vocab.index('woman')], size=20)

# ax.annotate('brother', X_tsne[vocab.index('brother')], size=20)
# ax.annotate('sister', X_tsne[vocab.index('sister')], size=20)

ax.annotate('can', X_tsne[vocab.index('can')], size=20)
ax.annotate('could', X_tsne[vocab.index('could')], size=20)
ax.annotate('must', X_tsne[vocab.index('must')], size=20)
ax.annotate('would', X_tsne[vocab.index('would')], size=20)

ax.annotate('feel', X_tsne[vocab.index('feel')], size=20)
ax.annotate('hurt', X_tsne[vocab.index('hurt')], size=20)


# ax.annotate('cheese', X_tsne[vocab.index('cheese')], size=20)
# ax.annotate('wine', X_tsne[vocab.index('wine')], size=20)
for word, pos in df.iterrows():
    if random.randint(0, 1) == 0 and False:
        ax.annotate(word, pos)

plt.title('Проекции векторов CBOW', size=20)

In [ ]:
glove_vectors.word_vec('person')

In [ ]:
with open('drive/MyDrive/Coursework2021/sticked_file.txt', 'r') as f:
    sentences = f.read().split('\n')
with open('drive/MyDrive/Coursework2021/big_eng_lstm_2000.txt', 'r') as f:
    sentences_g = f.read().split('\n')

In [ ]:
# ПРЕПРОЦЕССИНГ ВЕКТОРОВ ДЛЯ ПРЕДЛОЖЕНИЙ РЕАЛЬНЫХ И ПРЕДЛОЖЕНИЙ СИНТЕТИЧЕСКИХ

preprocessing_funcs = [
    str.lower,
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    strip_numeric,
    # remove_stopwords,
]

words_in_data = []
embeddings = []
bad_words = []

words_in_data_g = []
embeddings_g = []
bad_words_g = []

for sentence in tqdm(sentences):
    words = preprocess_string(sentence, preprocessing_funcs)
    for word in words:
        try:
            embedding = glove_vectors.word_vec(word)
            embeddings.append(embedding)
            words_in_data.append(word)
        except KeyError:
            bad_words.append(word)
            embeddings.append(glove_vectors.word_vec('token'))
            # print(word, '\n', sentence, '\n\n')


for sentence in tqdm(sentences_g):
    words = preprocess_string(sentence, preprocessing_funcs)
    for word in words:
        try:
            embedding = glove_vectors.word_vec(word)
            embeddings_g.append(embedding)
            words_in_data.append(word)
        except KeyError:
            bad_words_g.append(word)
            embeddings_g.append(glove_vectors.word_vec('token'))
            # print(word, '\n', sentence, '\n\n')

In [ ]:
print('ratio of bad words', len(bad_words) / len(embeddings), f'total {len(bad_words)} out of {len(embeddings)}')
print('ratio of bad words', len(bad_words_g) / len(embeddings_g), f'total {len(bad_words_g)} out of {len(embeddings_g)}')

In [ ]:
np.save('drive/MyDrive/Coursework2021/glove_embeddings_real.npy', np.asarray(embeddings[:10**6]))
np.save('drive/MyDrive/Coursework2021/glove_embeddings_gen.npy', np.asarray(embeddings_g[:10**6]))

In [ ]:
train_words = words_in_data[10**6: 10**6 + 1_000_000]
train_embeddings = np.vstack(embeddings[10**6:10**6 + 1_000_000])
# np.save('drive/MyDrive/Coursework2021/glove_preptrain_train_embeddings.npy', train_embeddings)

In [ ]:
print(len(set(train_words)))
Counter(train_words).most_common(10)

## Автоенкодер для GLOVE

In [ ]:
from keras.layers import InputLayer, Dense
from keras.models import Model, Sequential

INPUT_DIM = 50
OUTPUT_DIM = 16
ACTIVATION = 'LeakyReLU'

encoder = Sequential([
    InputLayer(input_shape=(INPUT_DIM,)),
    Dense(OUTPUT_DIM, activation=ACTIVATION)
])
decoder = Sequential([
    InputLayer(input_shape=(OUTPUT_DIM,)),
    Dense(INPUT_DIM, activation=None)
])
autoencoder = Model(inputs=encoder.input, outputs=decoder(encoder.output))
autoencoder.compile(loss='mse', optimizer='adam')

In [ ]:
h = autoencoder.fit(train_embeddings, train_embeddings, batch_size=16, epochs=6, verbose=True, validation_split=0.1)

print('testing')
encoded = autoencoder(np.vstack(embeddings[:10**6]))
print(mse(encoded, np.vstack(embeddings[:10**6])))

## Кластеризация сэмпла

### Класс Уишарта и его валидации

In [ ]:
from collections import defaultdict
import scipy
import numpy as np
import sys

from itertools import  product
from scipy.special import gamma
from scipy.spatial.distance import  euclidean
from math import sqrt

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KDTree

from sklearn.metrics import  silhouette_score, davies_bouldin_score, calinski_harabasz_score

import gc

def volume(r, m):
    return np.pi ** (m / 2) * r ** m / gamma(m / 2 + 1)

def significant(cluster, h, p):
    max_diff = max(abs(p[i] - p[j]) for i, j in product(cluster, cluster))

    # print(max_diff)
    return max_diff >= h


class WishartClusterization:
    def __init__(self, wishart_neighbors, significance_level):
        self.wishart_neighbors = wishart_neighbors  # Number of neighbors
        self.significance_level = significance_level  # Significance level

    def fit(self, X):
        kdt = KDTree(X, metric='euclidean')
        print('KDTree has been built')
        #add one because you are your neighb.
        distances, neighbors = kdt.query(X, k = self.wishart_neighbors + 1, return_distance = True)
        neighbors = neighbors[:, 1:]


        distances = distances[:, -1]
        print('distances')
        print(' '.join(list(map(str, distances))))


        indexes = np.argsort(distances)
        print('indexes sorted')
        print(' '.join(list(map(str, indexes))))

        size, dim = X.shape

        self.object_labels = np.zeros(size, dtype = int) - 1

        #index in tuple
        #min_dist, max_dist, flag_to_significant
        self.clusters = np.array([(1., 1., 0)])
        self.clusters_to_objects = defaultdict(list)
        #print('Start clustering')
        counter_to_print = 0
        for index in indexes:

            if counter_to_print % 100000 == 0:
                print(counter_to_print)
                print(' '.join(list(map(str, self.object_labels))), flush=True)

            counter_to_print += 1

            neighbors_clusters =\
                np.concatenate([self.object_labels[neighbors[index]], self.object_labels[neighbors[index]]])
            unique_clusters = np.unique(neighbors_clusters).astype(int)
            unique_clusters = unique_clusters[unique_clusters != -1]


            if len(unique_clusters) == 0:
                self._create_new_cluster(index, distances[index])
            else:
                max_cluster = unique_clusters[-1]
                min_cluster = unique_clusters[0]
                if max_cluster == min_cluster:
                    if self.clusters[max_cluster][-1] < 0.5:
                        self._add_elem_to_exist_cluster(index, distances[index], max_cluster)
                    else:
                        self._add_elem_to_noise(index)
                else:
                    my_clusters = self.clusters[unique_clusters]
                    flags = my_clusters[:, -1]
                    if np.min(flags) > 0.5:
                        self._add_elem_to_noise(index)
                    else:
                        significan = np.power(my_clusters[:, 0], -dim) - np.power(my_clusters[:, 1], -dim)
                        significan *= self.wishart_neighbors
                        significan /= size
                        significan /= np.power(np.pi, dim / 2)
                        significan *= gamma(dim / 2 + 1)
                        significan_index = significan >= self.significance_level

                        significan_clusters = unique_clusters[significan_index]
                        not_significan_clusters = unique_clusters[~significan_index]
                        significan_clusters_count = len(significan_clusters)
                        if significan_clusters_count > 1 or min_cluster == 0:
                            self._add_elem_to_noise(index)
                            self.clusters[significan_clusters, -1] = 1
                            for not_sig_cluster in not_significan_clusters:
                                if not_sig_cluster == 0:
                                    continue

                                for bad_index in self.clusters_to_objects[not_sig_cluster]:
                                    self._add_elem_to_noise(bad_index)
                                self.clusters_to_objects[not_sig_cluster].clear()
                        else:
                            for cur_cluster in unique_clusters:
                                if cur_cluster == min_cluster:
                                    continue

                                for bad_index in self.clusters_to_objects[cur_cluster]:
                                    self._add_elem_to_exist_cluster(bad_index, distances[bad_index], min_cluster)
                                self.clusters_to_objects[cur_cluster].clear()

                            self._add_elem_to_exist_cluster(index, distances[index], min_cluster)
        self.labels_ = self.clean_data()
        return self.labels_

    def clean_data(self):
        unique = np.unique(self.object_labels)
        index = np.argsort(unique)
        if unique[0] != 0:
            index += 1
        true_cluster = {unq :  index for unq, index in zip(unique, index)}
        result = np.zeros(len(self.object_labels), dtype = int)
        for index, unq in enumerate(self.object_labels):
            result[index] = true_cluster[unq]
        return result

    def _add_elem_to_noise(self, index):
        self.object_labels[index] = 0
        self.clusters_to_objects[0].append(index)

    def _create_new_cluster(self, index, dist):
        self.object_labels[index] = len(self.clusters)
        self.clusters_to_objects[len(self.clusters)].append(index)
        self.clusters = np.append(self.clusters, [(dist, dist, 0)], axis = 0)

    def _add_elem_to_exist_cluster(self, index, dist, cluster_label):
        self.object_labels[index] = cluster_label
        self.clusters_to_objects[cluster_label].append(index)
        self.clusters[cluster_label][0] = min(self.clusters[cluster_label][0], dist)
        self.clusters[cluster_label][1] = max(self.clusters[cluster_label][1], dist)

In [ ]:
!git clone https://github.com/jqmviegas/jqm_cvi.git
!wget https://raw.githubusercontent.com/iphysresearch/S_Dbw_validity_index/master/S_Dbw.py

Cloning into 'jqm_cvi'...
remote: Enumerating objects: 92, done.
remote: Total 92 (delta 0), reused 0 (delta 0), pack-reused 92
Unpacking objects: 100% (92/92), done.
--2021-06-05 18:59:39--  https://raw.githubusercontent.com/iphysresearch/S_Dbw_validity_index/master/S_Dbw.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4235 (4.1K) [text/plain]
Saving to: ‘S_Dbw.py’

S_Dbw.py            100%[===================>]   4.14K  --.-KB/s    in 0s      

2021-06-05 18:59:39 (55.6 MB/s) - ‘S_Dbw.py’ saved [4235/4235]



In [ ]:
import numpy as np
import pandas as pd

from scipy.spatial.distance import pdist, squareform
from scipy.special import gamma
from collections import defaultdict

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import cycle, islice
import seaborn as sns
from sklearn.metrics import adjusted_rand_score, silhouette_score
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from jqm_cvi.jqmcvi import base
from S_Dbw import S_Dbw
from sklearn.neighbors import NearestNeighbors

# utils
def volume(r, dim):
    """
    Helper function to calculate volumes of a dim-dimensional spheres with radiuses r
    param r: radiuses of a spheres of shape (n_samples,): ndarray
    param dim: dimensionality of a sphere: int

    Returns:
        volumes: ndarray
    """
    dim_const = (np.pi ** (dim / 2)) / gamma(dim / 2 + 1)
    return dim_const * (r ** dim)


def significant(cluster, h, p):
    """
    Helper function to tell if a cluster is significant
    param cluster: vertices in cluster (indexed): list
    h: height hyperparameter: double
    p: estimated saliency for all points
    """
    p_cluster = p[np.array(cluster)]
    pw_difference = np.abs(p_cluster[:, np.newaxis] - p_cluster)
    max_diff = pw_difference.max()

    # print(max_diff)
    return max_diff >= h


class ClusterValidation:
    def __init__(self, data, labels, dist='euclidean'):
        """
        @param data: ndarray of shape (n_samples, n_features)
        @param labels: ndarray of shape (n_samples,), cluster labels for each sample
        @param dist: distance metric to use: str or callable; defaults to 'euclidean'
        """
        self.data = data
        self.labels = labels
        self.dist = dist
        self.num_cluster = labels.max() + 1
        self.cluster_list = list(range(self.num_cluster))

    def RMSSTD(self):
        dof = self.data.shape[0] - self.num_cluster
        n_features = self.data.shape[1]
        result = 0
        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            center = cluster_data.mean(axis=0)
            result += ((cluster_data - center) ** 2).sum()

        return (result / (dof * n_features)) ** (1 / 2)

    def RS(self):
        all_mean = ((self.data - self.data.mean(axis=0)) ** 2).sum()
        part_mean = 0

        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            center = cluster_data.mean(axis=0)
            part_mean += ((cluster_data - center) ** 2).sum()

        return (all_mean - part_mean) / all_mean

    def hubert(self):
        dist_matrix = squareform(pdist(self.data, self.dist))
        n = self.data.shape[0]

        centers = np.empty((self.num_cluster, self.data.shape[1]))
        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            center = cluster_data.mean(axis=0)
            centers[cluster] = center
        center_dist = squareform(pdist(centers, self.dist))

        result = 0
        for i in range(n):
            for j in range(n):
                result += (dist_matrix[i, j] * center_dist[self.labels[i], self.labels[j]])
        return (result * 2) / (n * (n-1)) 

    def CH(self):
        return calinski_harabasz_score(self.data, self.labels)

    def I_index(self):
        all_mean = np.sqrt(((self.data - self.data.mean(axis=0)) ** 2).sum())
        part_mean = 0
        centers = np.empty((self.num_cluster, self.data.shape[1]))

        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            center = cluster_data.mean(axis=0)
            centers[cluster] = center
            part_mean += np.sqrt(((cluster_data - center) ** 2).sum())
        center_dist = squareform(pdist(centers, self.dist))

        max_center_dist = center_dist.max()

        return (all_mean * max_center_dist) / (self.num_cluster * part_mean)

    def dunn(self):
        return base.dunn_fast(self.data, self.labels)

    def silhouette(self):
        return silhouette_score(self.data, self.labels)

    def davies_bouldin(self):
        return davies_bouldin_score(self.data, self.labels)

    def xie_beni(self):
        n = self.data.shape[0]
        part_mean = 0

        centers = np.empty((self.num_cluster, self.data.shape[1]))
        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            center = cluster_data.mean(axis=0)
            centers[cluster] = center
            part_mean += ((cluster_data - center) ** 2).sum()
        center_dist = squareform(pdist(centers, self.dist))

        center_dist = center_dist + np.diag(np.full_like(center_dist[0], center_dist.max()))

        return part_mean / (n * (center_dist.min()**2))

    def SD(self):
        scat, dis = 0, 0

        all_var_vector = np.var(self.data, 0)
        all_var = np.sqrt(np.sum(all_var_vector ** 2))
        part_var = 0

        centers = np.empty((self.num_cluster, self.data.shape[1]))
        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            center = cluster_data.mean(axis=0)
            centers[cluster] = center
            part_var_vector = np.var(cluster_data, 0)
            part_var += np.sqrt((part_var_vector ** 2).sum())
        center_dist = squareform(pdist(centers, self.dist))

        scat = part_var / (all_var * self.num_cluster)

        max_dist = center_dist.max()
        min_dist = (center_dist + np.diag(np.full_like(center_dist[0], max_dist))).min()
        sum_dist = np.sum(1.0 / np.sum(center_dist, axis=1))

        dis = (max_dist * sum_dist) / min_dist

        return scat + dis

    def S_Dbw(self):
        centers = np.empty((self.num_cluster, self.data.shape[1]))
        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            center = cluster_data.mean(axis=0)
            centers[cluster] = center

        sdbw = S_Dbw(self.data, self.labels, centers)
        return sdbw.S_Dbw_result()

    def CVNN(self, n_neighbors=10):
        com = 0
        sep = []
        neigh = NearestNeighbors(n_neighbors=n_neighbors)
        neigh.fit(self.data)
        knbrs = neigh.kneighbors(return_distance=False)

        for cluster in self.cluster_list:
            cluster_data = self.data[self.labels == cluster, :]
            cluster_ind = np.arange(self.data.shape[0])[self.labels == cluster]
            n_i = cluster_data.shape[0]
            if (n_i > 1):
                com += np.sum(pdist(cluster_data, metric=self.dist)) * 2 / (n_i * (n_i-1))
            sep.append(np.isin(knbrs[cluster_ind], cluster_ind, invert=True).sum() / (n_i * n_neighbors))

        sep = np.max(sep)

        return com + sep

    def run_all(self, n_neighbors=10, calc_hubert=True):
        results = {}
        results['rmsstd'] = self.RMSSTD()
        results['r-squared'] = self.RS()
        if calc_hubert:
            results['hubert'] = self.hubert()
        results['ch_index'] = self.CH()
        results['i_index'] = self.I_index()
        try:
            results['dunn'] = self.dunn()
        except ValueError as e:
            print('dunn failed with')
            print(e)
            results['dunn'] = np.nan
        results['silhouette'] = self.silhouette()
        results['davies_bouldin'] = self.davies_bouldin()
        results['xie_beni'] = self.xie_beni()
        results['sd'] = self.SD()
        try:
            results['s_dbw'] = self.S_Dbw()
        except:
            results['s_dbw'] = np.nan
        results['cvnn'] = self.CVNN(n_neighbors)

        return results


1.2260036414029734


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
from scipy.stats import ranksums, kstest, ks_2samp
from scipy.spatial.distance import cdist

def cluster_mean_intracluster(X, y):
    mean_intracluster_dist = []
    for cluster_id in range(len(np.unique(y))):
        X_cluster = X[y == cluster_id]
        if len(X_cluster) < 2:
            mean_intracluster_dist.append(0)
            continue
        cluster_dist = squareform(pdist(X_cluster))
        n = X_cluster.shape[0]
        mean_intracluster_dist.append(2 * cluster_dist.sum() / (n * (n - 1)))
    return mean_intracluster_dist

def cluster_max_intracluster(X, y):
    max_intracluster_dist = []
    for cluster_id in range(len(np.unique(y))):
        X_cluster = X[y == cluster_id]
        if len(X_cluster) < 2:
            continue
        cluster_dist = squareform(pdist(X_cluster))
        max_intracluster_dist.append(cluster_dist.max())
    return max_intracluster_dist

def cluster_centroid_dist(X, y):
    centroid_dist = []
    for cluster_id in range(len(np.unique(y))):
        X_cluster = X[y == cluster_id]
        if len(X_cluster) < 2:
            continue
        centroid = np.mean(X_cluster, axis=0)
        c_dist = cdist(X_cluster, centroid.reshape(1, -1))
        centroid_dist.append(2 * c_dist.mean())
    return centroid_dist

def cluster_rmsstd(X, y):
    rmsstd_centroid_dist = []
    dof = X.shape[0] - len(np.unique(y))
    n_feat = X.shape[1]
    for cluster_id in range(len(np.unique(y))):
        X_cluster = X[y == cluster_id]
        if len(X_cluster) < 2:
            continue
        centroid = np.mean(X_cluster, axis=0)
        centroid_dist = cdist(X_cluster, centroid.reshape(1, -1))
        rmsstd_centroid_dist.append((centroid_dist.sum() / (dof * n_feat)) ** (0.5))

    return rmsstd_centroid_dist

def cluster_sd_scat(X, y):
    sd_scat = []
    all_var_vector = np.var(X, 0)
    all_var = np.sqrt(np.sum(all_var_vector ** 2))
    for cluster_id in range(len(np.unique(y))):
        X_cluster = X[y == cluster_id]
        if len(X_cluster) < 2:
            continue
        part_var_vector = np.var(X_cluster, 0)
        sd_scat.append(np.sqrt((part_var_vector ** 2).sum()))
    return sd_scat


def cluster_silhouette(X, y):
    result = []
    n_clusters = len(np.unique(y))
    for cluster_id in range(n_clusters):
        X_cluster = X[y == cluster_id]
        if len(X_cluster) < 2:
            continue
        
        cluster_dist = squareform(pdist(X_cluster))
        n = X_cluster.shape[0]
        a = cluster_dist.sum(axis=1) / (n - 1)
        # other clusters
        b = None

        for other_cluster_id in range(n_clusters):
            if other_cluster_id == cluster_id:
                continue
            X_other_cluster = X[y == other_cluster_id]
            between_cluster_dist = cdist(X_cluster, X_other_cluster)
            mean_between = between_cluster_dist.mean(axis=1)

            if b is None:
                b = mean_between
            else:
                b = np.minimum(b, mean_between)
        
        result.append(np.mean((b - a) / np.maximum(a, b)) / n_clusters)

    return result

def cluster_db(X, y):
    n_clusters = len(np.unique(y))
    intra_dists = np.empty(n_clusters)
    centroids = np.empty((n_clusters, len(X[0])), dtype=float)

    for cluster_id in range(n_clusters):
        X_cluster = X[y == cluster_id]
        centroid = np.mean(X_cluster, axis=0)
        centroids[cluster_id] = centroid
        intra_dists[cluster_id] = np.mean(cdist(X_cluster, centroid.reshape(1, -1)))

    centroid_dist = squareform(pdist(centroids))

    centroid_dist[centroid_dist == 0] = np.inf
    intra_dists_between_clusters = intra_dists[:, None] + intra_dists
    scores = np.max(intra_dists_between_clusters / centroid_dist, axis=1)

    return scores

from sklearn.neighbors import NearestNeighbors
def cluster_cvnn_sep(X, y, n_neighbors=10):
    n_clusters = len(np.unique(y))
    neigh = NearestNeighbors(n_neighbors=n_neighbors)
    neigh.fit(X)
    knbrs = neigh.kneighbors(return_distance=False)

    sep = []

    for cluster_id in range(n_clusters):
        X_cluster = X[y == cluster_id]
        cluster_ind = np.arange(X.shape[0])[y == cluster_id]
        n_i = X_cluster.shape[0]
        
        sep.append(np.isin(knbrs[cluster_ind], cluster_ind, invert=True).sum() / (n_i * n_neighbors))

    return sep

def run_all_stat_tests(X_real, y_real, X_gen, y_gen):
    wilcoxon_res = {}
    ks_res = {}
    fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(15, 15))

    # sizes
    _, real_sizes = np.unique(y_real, return_counts=True)
    _, gen_sizes = np.unique(y_gen, return_counts=True)
    wilcoxon_res['sizes'] = ranksums(real_sizes, gen_sizes)
    ks_res['sizes'] = ks_2samp(real_sizes, gen_sizes)

    sns.kdeplot(x=real_sizes, label='real', ax=axs[0][0])
    sns.kdeplot(x=gen_sizes, label='gen', ax=axs[0][0])
    axs[0][0].set_title('Sizes of clusters')
    axs[0][0].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['sizes'].pvalue,
                       ks_res['sizes'].pvalue),
                   transform=axs[0][0].transAxes)
    axs[0][0].legend()

    # mean intracluster
    stat_real = cluster_mean_intracluster(X_real, y_real)
    stat_gen = cluster_mean_intracluster(X_gen, y_gen)
    wilcoxon_res['mean_dist'] = ranksums(stat_real, stat_gen)
    ks_res['mean_dist'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[0][1])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[0][1])
    axs[0][1].set_title('Mean intracluster distance')
    axs[0][1].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['mean_dist'].pvalue,
                       ks_res['mean_dist'].pvalue),
                   transform=axs[0][1].transAxes)
    axs[0][1].legend()

    # max intracluster
    stat_real = cluster_max_intracluster(X_real, y_real)
    stat_gen = cluster_max_intracluster(X_gen, y_gen)
    wilcoxon_res['max_dist'] = ranksums(stat_real, stat_gen)
    ks_res['max_dist'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[0][2])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[0][2])
    axs[0][2].set_title('Max intracluster distance')
    axs[0][2].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['max_dist'].pvalue,
                       ks_res['max_dist'].pvalue),
                   transform=axs[0][2].transAxes)
    axs[0][2].legend()

    # centroid dist
    stat_real = cluster_centroid_dist(X_real, y_real)
    stat_gen = cluster_centroid_dist(X_gen, y_gen)
    wilcoxon_res['centroid_dist'] = ranksums(stat_real, stat_gen)
    ks_res['centroid_dist'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[1][0])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[1][0])
    axs[1][0].set_title('Centroid distance')
    axs[1][0].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['centroid_dist'].pvalue,
                       ks_res['centroid_dist'].pvalue),
                   transform=axs[1][0].transAxes)
    axs[1][0].legend()

    # rmsstd compactness
    stat_real = cluster_rmsstd(X_real, y_real)
    stat_gen = cluster_rmsstd(X_gen, y_gen)
    wilcoxon_res['rmsstd'] = ranksums(stat_real, stat_gen)
    ks_res['rmsstd'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[1][1])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[1][1])
    axs[1][1].set_title('RMSSTD Compactness')
    axs[1][1].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['rmsstd'].pvalue,
                       ks_res['rmsstd'].pvalue),
                   transform=axs[1][1].transAxes)
    axs[1][1].legend()

    # sd scattering
    stat_real = cluster_sd_scat(X_real, y_real)
    stat_gen = cluster_sd_scat(X_gen, y_gen)
    wilcoxon_res['sd_scat'] = ranksums(stat_real, stat_gen)
    ks_res['sd_scat'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[1][2])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[1][2])
    axs[1][2].set_title('SD scattering')
    axs[1][2].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['sd_scat'].pvalue,
                       ks_res['sd_scat'].pvalue),
                   transform=axs[1][2].transAxes)
    axs[1][2].legend()

    # silhouette index
    stat_real = cluster_silhouette(X_real, y_real)
    stat_gen = cluster_silhouette(X_gen, y_gen)
    wilcoxon_res['silhouette'] = ranksums(stat_real, stat_gen)
    ks_res['silhouette'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[2][0])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[2][0])
    axs[2][0].set_title('Silhouette index')
    axs[2][0].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['silhouette'].pvalue,
                       ks_res['silhouette'].pvalue),
                   transform=axs[2][0].transAxes)
    axs[2][0].legend()

    # db cluster scores
    stat_real = cluster_db(X_real, y_real)
    stat_gen = cluster_db(X_gen, y_gen)
    wilcoxon_res['db_score'] = ranksums(stat_real, stat_gen)
    ks_res['db_score'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[2][1])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[2][1])
    axs[2][1].set_title('DB cluster scores')
    axs[2][1].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['db_score'].pvalue,
                       ks_res['db_score'].pvalue),
                   transform=axs[2][1].transAxes)
    axs[2][1].legend()

    # cvnn separation
    stat_real = cluster_cvnn_sep(X_real, y_real)
    stat_gen = cluster_cvnn_sep(X_gen, y_gen)
    wilcoxon_res['cvnn_sep'] = ranksums(stat_real, stat_gen)
    ks_res['cvnn_sep'] = ks_2samp(stat_real, stat_gen)

    sns.kdeplot(x=stat_real, label='real', ax=axs[2][2])
    sns.kdeplot(x=stat_gen, label='gen', ax=axs[2][2])
    axs[2][2].set_title('CVNN separation')
    axs[2][2].text(0.01, 0.9,
                   'Wilcoxon pvalue = {:.4f}\nKS pvalue = {:.4f}'.format(
                       wilcoxon_res['cvnn_sep'].pvalue,
                       ks_res['cvnn_sep'].pvalue),
                   transform=axs[2][2].transAxes)
    axs[2][2].legend()

    # plt.tight_layout()
    plt.show()
    return wilcoxon_res, ks_res

### Результаты

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(16)
pca.fit(embeddings[10**6:10**6 + 500_000])

In [ ]:
def get_ngrams_from_vectors(X, n=3):
    """
    :param X: данные (n_samples, n_features)
    :param n: количество векторов в n-грамме
    :return: массив уникальных n-грамм
    """
    X = np.asarray(X)
    ngrams = []
    for i in range(len(X) - n + 1):
        # ngrams.append(encoder(np.asarray(X[i:i + n])).numpy().flatten())
        ngrams.append(pca.transform(np.asarray(X[i:i + n])).flatten())
    return np.vstack(ngrams)

In [ ]:
ngram_series = get_ngrams_from_vectors(embeddings[:5000])
ngram_series.shape

In [ ]:
wishart = WishartClusterization(2, 1)
labels = wishart.fit(ngram_series)
labels

In [ ]:
Counter(labels).most_common(15)

In [ ]:
ngram_series_g = get_ngrams_from_vectors(embeddings_g[:5000])
ngram_series_g.shape

wishart_g = WishartClusterization(2, 1)
labels_g = wishart_g.fit(ngram_series_g)
Counter(labels_g).most_common()

In [ ]:
cv_real = ClusterValidation(ngram_series_g, labels_g)
cv_real.run_all()

In [ ]:
fig, _, _ = run_all_stat_tests(ngram_series, labels, ngram_series_g, labels_g)

# Предобученный FastText

In [ ]:
fasttext_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')

In [ ]:
preprocessing_funcs = [
    str.lower,
    strip_tags,
    strip_punctuation,
    strip_multiple_whitespaces,
    strip_numeric,
    # remove_stopwords,
]

words_in_data = []
embeddings = []
bad_words = []

for sentence in tqdm(sentences):
    words = preprocess_string(sentence, preprocessing_funcs)
    for word in words:
        try:
            embedding = fasttext_vectors.word_vec(word)
            embeddings.append(embedding)
            words_in_data.append(word)
        except KeyError:
            bad_words.append(word)
            # embeddings.append(fasttext_vectors.word_vec('token'))

In [ ]:
print('ratio of bad words', len(bad_words) / len(embeddings))

## Автоенкодер для FastText

In [ ]:
from keras.layers import InputLayer, Dense
from keras.models import Model, Sequential

INPUT_DIM = 300
OUTPUT_DIM = 40
ACTIVATION = 'tanh'

encoder = Sequential([
    InputLayer(input_shape=(INPUT_DIM,)),
    Dense(OUTPUT_DIM, activation=ACTIVATION)
])
decoder = Sequential([
    InputLayer(input_shape=(OUTPUT_DIM,)),
    Dense(INPUT_DIM, activation=None)
])
autoencoder = Model(inputs=encoder.input, outputs=decoder(encoder.output))
autoencoder.compile(loss='mse', optimizer='adam')

In [ ]:
train_words = words_in_data[10**6: 10**6 + 1_000_000]
train_embeddings = np.vstack(embeddings[10**6:10**6 + 1_000_000])

print(len(set(train_words)))
Counter(train_words).most_common(10)

In [ ]:
h = autoencoder.fit(train_embeddings, train_embeddings, batch_size=16, epochs=10, verbose=True, validation_split=0.1)

## Кластеризация сэмпла

# Кластеризация своего w2v

In [ ]:
model32_u = gensim.models.KeyedVectors.load_word2vec_format('drive/MyDrive/Coursework2021/cbow/mincnt10_window5_size32_unnormed.kv')

In [ ]:
sentences = []
word_cnt = 0

with open('drive/MyDrive/Coursework2021/big_eng_lstm_2000.txt', 'r') as f:
    for line in f:
        sentences.append(line)
        word_cnt += len(line.split())

In [ ]:
word_cnt

1685238